# Homework 2: **Bayesian** *regression* and *classification*, ***Gaussian Processes*** (part 1)

Probabilistic Machine Learning -- Spring 2023, UniTS

<a target="_blank" href="https://colab.research.google.com/github/emaballarin/probml-units/blob/main/homework-2023/02_bayesian_reg_class_gps_HW.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

### Exercise 1 (theory: *Bayesian conjugate modelling*)

Consider the Gamma distribution $\text{Gamma}(\alpha, \beta)$ with p.d.f. $\frac{\beta^\alpha}{\Gamma(\alpha)}x^{\alpha-1}e^{-\beta x}$, and the Poisson distribution $\text{Poisson}(\lambda)$ with p.m.f. $\frac{\lambda^k e^{-\lambda}}{k!}.$

Given the model
$$
    \begin{align*}
    \lambda &\sim \text{Gamma}(\alpha, \beta)\\
    x &\sim \text{Poisson}(\lambda)
    \end{align*}
$$
prove that the Gamma distribution is a conjugate prior for the Poisson likelihood (for the given model).

### Exercise 2 (practicum: *`sklearn`, Naive Bayes*)

The *Wine* dataset is a classic *toy dataset* for classification tasks. It contains 178 samples of 3 different types of wine (encoded with a sequantial integer), with 13 continuous numeric features each. The goal is to (learn to) predict the type of an unknown wine (of any of those 3 types), given its specific features.

It can be easily imported from `sklearn` as:

```python
from sklearn.datasets import load_wine
... = load_wine()
```

Trying to use the `sklearn` interface as much as possible (no not *re-invent the wheel*!):

1. Split the dataset into a training and a test set (*hint:* use the `train_test_split` function), with the test split approximately $1/3^\text{rd}$ of the available data. Ensure that both sets are adequately representative of the problem at hand!

2. Fit a `LogisticRegression` model on the *training* split, and try to predict the labels from (just) the numeric features of the *test* split (*note:* we are not dealing with a *Bayesian* classifier yet; that will require - apart from theoretical knowledge, which you should already have! - sufficient experience with a Bayesian statistical modelling library, *e.g.* [Pyro](https://pyro.ai/), which will be introduced to you soon!)

3. Evaluate the performance of the model on the *test* split.

4. Fit a `GaussianNB` (*Naive Bayes classifier*) model on the *training* split, and try to predict the labels from (just) the numeric features of the *test* split. Evaluate the performance of the model on the *test* split.

5. Compare the performance of the two models according to a suitable metric and briefly comment.

### Exercise 3 (practicum: *`sklearn`, Gaussian Process regression fitting*)

The *Mauna Loa Carbon Dioxide* dataset is another *toy* dataset, this time for **regression tasks** (specifically *time series* regression and prediction). It contains the raw recordings of atmospheric CO$_2$ concentration (in parts per million) at the Mauna Loa Observatory in Hawaii, in some days between 1958 and 2001. The goal is to (learn to) predict the CO$_2$ concentration at a given time, given the previous measurements.

By using some clever `sklearn` utilities, it can be easily imported as:

```python
from sklearn.datasets import fetch_openml
... = fetch_openml(data_id=41187, as_frame=True, parser="pandas")
```

On the resulting dataset, perform the following preprocessing steps (in `pandas`):

1. Drop from the dataset all the columns with data other than time information (if you want to convert time information to proper dates, you can use the `pandas.to_datetime` function; but that;s up to you!) and the actual carbon dioxide concentration (`co2`).

2. Compute monthly averages of the `co2` column and remove from the dataframe all resulting rows (*i.e.* months with no measurements al all).

In [1]:
# YOUR CODE HERE!

Exploiting the contents of `sklearn.gaussian_process.kernels`, design an adequate kernel for the (predictive) regression task at hand, and fit a `GaussianProcessRegressor` model on a *training* split (which you need to define from original data, approximately 0.85 of the whole dataset), and try to predict the labels from (just) the numeric features of the *test* split. Evaluate the performance of the model on the remaining *test* split.

Notice that the task has been already solved (many!) times before (and such solutions may be available online!). The point of the exercise is to gain some experience with the `sklearn` interface for Gaussian Processes regression, and to get a feel for the kind of effects modelled by different choices of kernel (not to get perfect results!).

If you want to try (and show!) different possible solutions, feel free to do it. You may discover some potentially interesting phenomena!

In [2]:
# YOUR CODE HERE!

**OPTIONAL:**  
Try to fit the model on the whole dataset and predict values of carbon dioxide concentration until today. Does the model capture the features of the time series well enough?

### Exercise 4 (practicum: `sklearn`, Gaussian Process classification fitting; potentially requiring theory *to-be-covered*)

Consider an instance of the *Circles* dataset, yet another *toy* dataset for classification featuring two concentric elliptical annuli. For the sake of simplicity, it has been already be generated and plotted for you:

In [3]:
import numpy as np
from sklearn.datasets import make_circles
from matplotlib import pyplot as plt

In [4]:
x, y = make_circles(n_samples=1000, factor=0.3, noise=0.05, random_state=0)
_ = plt.scatter(x[:, 0], x[:, 1], c=y)

`sklearn`'s `GaussianProcessClassifier` approximates the non-Gaussian posterior by a Gaussian approximated through Laplace's saddle-point approximation.

Define an RBF kernel (*hint*: there's a `sklearn.gaussian_process.kernels.RBF` function just for that!) with lenghtscale parameter $l = 1$ and fit a Gaussian Process classifier to the observed data $(x,y)$ from *Circles* right above.

In [5]:
# YOUR CODE HERE!

The functions that follow, evaluate and plot the posterior predictive mean function over a finite grid of points. Most importantly, the `plot_sklearn_predictions` function requires as arguments the learned GP classifier `sklearn_gp_classifier`, the observed points `x` and their labels `y`.

In [6]:
def meshgrid(x, n, eps=0.1):
    x0, x1 = np.meshgrid(
        np.linspace(x[:, 0].min() - eps, x[:, 0].max() + eps, n),
        np.linspace(x[:, 1].min() - eps, x[:, 1].max() + eps, n),
    )
    x_grid = np.stack([x0.ravel(), x1.ravel()], axis=-1)
    return x0, x1, x_grid


def plot_sklearn_predictions(sklearn_gp_classifier, x, y):
    x0, x1, x_grid = meshgrid(x, 30)

    preds = sklearn_gp_classifier.predict_proba(x_grid)

    _ = plt.figure(figsize=(10, 6))
    _ = plt.contourf(x0, x1, preds[:, 0].reshape(x0.shape), 101, vmin=0, vmax=1)
    _ = plt.contourf(x0, x1, preds[:, 1].reshape(x0.shape), 101, vmin=0, vmax=1)

    _ = plt.title("Posterior Mean")
    _ = plt.xticks([])
    _ = plt.yticks([])
    _ = plt.colorbar()
    _ = plt.scatter(x[:, 0], x[:, 1], c=y)

Plot the posterior predictive mean function and the training data, and comment.

In [7]:
# YOUR CODE HERE!

### Not-an-exercise 5 (*Kernels? Why bother?*)

The following *task* is completely optional and up to your personal curiosity (and also a result of *my own* curiosity 🙃). It will be in no way considered for evaluation-related purposes: it may lead to interesting discussions though (or exam projects? and beyond? who knows!).

If you have never had the occasion to, watch this [brief talk by Ali Rahimi](https://www.youtube.com/watch?v=ORHFOnaEzPc), given in occasion of the 2018 *NeurIPS Test of Time award* (awarded his co-authored 2007 paper with Ben Recht ["Random Features for Large-Scale Kernel Machines"](https://people.eecs.berkeley.edu/~brecht/papers/07.rah.rec.nips.pdf)).

If you have some comments about it, feel free to write or voice them!